In [58]:
import pandas as pd
import numpy as np

In [59]:
data = pd.read_csv('Stock Market Dataset.csv')

In [60]:
data['Date'] = pd.to_datetime(data['Date'], dayfirst=True)
data.set_index('Date', inplace=True) 
data = data.iloc[:, 1:]
data

,Natural_Gas_Price,Natural_Gas_Vol.,Crude_oil_Price,Crude_oil_Vol.,Copper_Price,Copper_Vol.,Bitcoin_Price,Bitcoin_Vol.,Platinum_Price,Platinum_Vol.,...,Berkshire_Price,Berkshire_Vol.,Netflix_Price,Netflix_Vol.,Amazon_Price,Amazon_Vol.,Meta_Price,Meta_Vol.,Gold_Price,Gold_Vol.
Date,,,,,,,,,,,,,,,,,,,,,
2024-02-02,2.079,NaN,72.28,NaN,3.8215,NaN,"43,194.70",42650.0,901.6,NaN,...,"5,89,498",10580.0,564.64,4030000.0,171.81,117220000.0,474.99,84710000.0,"2,053.70",NaN
2024-02-01,2.050,161340.0,73.82,577940.0,3.8535,NaN,"43,081.40",47690.0,922.3,NaN,...,"5,81,600",9780.0,567.51,3150000.0,159.28,66360000.0,394.78,25140000.0,"2,071.10",260920.0
2024-01-31,2.100,142860.0,75.85,344490.0,3.9060,NaN,"42,580.50",56480.0,932.6,NaN,...,"5,78,020",9720.0,564.11,4830000.0,155.20,49690000.0,390.14,20010000.0,"2,067.40",238370.0
2024-01-30,2.077,139750.0,77.82,347240.0,3.9110,NaN,"42,946.20",55130.0,931.7,NaN,...,"5,84,680",9750.0,562.85,6120000.0,159.00,42290000.0,400.06,18610000.0,"2,050.90",214590.0
2024-01-29,2.490,3590.0,76.78,331930.0,3.8790,NaN,"43,299.80",45230.0,938.3,NaN,...,"5,78,800",13850.0,575.79,6880000.0,161.26,42840000.0,401.02,17790000.0,"2,034.90",1780.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-02-08,2.583,147880.0,52.72,621000.0,2.8140,270.0,"3,661.70",699230.0,802.2,NaN,...,"3,00,771",240.0,347.57,7560000.0,79.41,113150000.0,167.33,12560000.0,"1,318.50",150610.0
2019-02-07,2.551,211790.0,52.64,749010.0,2.8320,320.0,"3,397.70",471360.0,800.8,NaN,...,"3,02,813",240.0,344.71,7860000.0,80.72,92530000.0,166.38,17520000.0,"1,314.20",166760.0
2019-02-06,2.662,98330.0,54.01,606720.0,2.8400,100.0,"3,404.30",514210.0,807.1,NaN,...,"3,08,810",120.0,352.19,6720000.0,82.01,78800000.0,170.49,13280000.0,"1,314.40",137250.0


In [61]:
data.columns

Index(['Natural_Gas_Price', 'Natural_Gas_Vol.', 'Crude_oil_Price',
       'Crude_oil_Vol.', 'Copper_Price', 'Copper_Vol.', 'Bitcoin_Price',
       'Bitcoin_Vol.', 'Platinum_Price', 'Platinum_Vol.', 'Ethereum_Price',
       'Ethereum_Vol.', 'S&P_500_Price', 'Nasdaq_100_Price', 'Nasdaq_100_Vol.',
       'Apple_Price', 'Apple_Vol.', 'Tesla_Price', 'Tesla_Vol.',
       'Microsoft_Price', 'Microsoft_Vol.', 'Silver_Price', 'Silver_Vol.',
       'Google_Price', 'Google_Vol.', 'Nvidia_Price', 'Nvidia_Vol.',
       'Berkshire_Price', 'Berkshire_Vol.', 'Netflix_Price', 'Netflix_Vol.',
       'Amazon_Price', 'Amazon_Vol.', 'Meta_Price', 'Meta_Vol.', 'Gold_Price',
       'Gold_Vol.'],
      dtype='object')

In [62]:
sector_names = []
for var in data.columns:
    if var.endswith('_Price'):
        sector_names += [var.replace('_Price', '')]
    elif var.endswith('_Vol'):
        sector_names += [var.replace('_Vol', '')]
sector_names = list(set(sector_names))
sector_names

['Apple',
 'Netflix',
 'Natural_Gas',
 'Nasdaq_100',
 'Platinum',
 'S&P_500',
 'Silver',
 'Amazon',
 'Ethereum',
 'Microsoft',
 'Meta',
 'Bitcoin',
 'Berkshire',
 'Crude_oil',
 'Nvidia',
 'Google',
 'Gold',
 'Copper',
 'Tesla']

In [63]:
def cross_sectional_standardization(df, eps=1e-10):
        df = df.T
        df = (df - df.mean()) / (df.std()+eps)
        return df.T

In [64]:
def calculate_percentage_change(df, eps=1e-10):
    return ((df - df.shift(1))/df.shift(1)+eps)

In [99]:
def get_data(df, sector_names, suffix='_Price', pct=False, standardize=False, fill_na=True):
    sector_suffix_dict = {sector + suffix: sector for sector in sector_names}  # Gold_Price -> Gold
    valid_sectors = set (sector_suffix_dict) & set(df.columns)
    invalid_sectors = set (sector_suffix_dict) - set(df.columns)
    ret = df[list(valid_sectors)].copy()
    for sector in invalid_sectors:
        ret[sector] = np.nan
    ret.rename(columns=sector_suffix_dict, inplace=True)
    
    for sector in ret.select_dtypes(include=['object', 'string']).columns:
        ret[sector] = ret[sector].str.replace(',', '').astype(float)
    
    if pct:
        ret=calculate_percentage_change(ret)
    if standardize:
        ret = cross_sectional_standardization(ret)
    if fill_na:
        ret = ret.fillna(0)
    
    return ret

In [100]:
price_data = get_data(data, sector_names, suffix='_Price', pct=True, standardize=True, fill_na=True)
price_data.to_csv('price_data.csv')

In [101]:
vol_data = get_data(data, sector_names, suffix='_Vol.', pct=True, standardize=True, fill_na=True)
vol_data.to_csv('vol_data.csv')